Abhay Singh

Email: abhay.s-26@scds.saiuniversity.edu.in

School of Computing and Data Science

### Hutton Rock Dataset

####Total Number of images: 505

- The number of images in granite class are: 187
- The number of images in basalt class are: 130
- The number of images in coal class are: 85
- The number of images in andesite class are: 103

###378 images are used for trining and 127 images are used for testing.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
print(tf.__version__)

from tensorflow import keras
tf.random.set_seed(42)

import numpy as np
np.random.seed(42)

import matplotlib.pyplot as plt
%matplotlib inline

import glob
import PIL
from PIL import Image

2.17.1


In [3]:
imgFiles = glob.glob("/content/drive/MyDrive/DLPROJECT/Hutton_Rock/*/*.jpg")

In [4]:
print(len(imgFiles))

508


In [5]:
X = []
y = []

for fName in imgFiles:

  # Prepare the dataset and populate X and y
  X_i = Image.open(fName)
  X_i = X_i.resize((299,299))
  if X_i.mode != 'RGB':
    X_i = X_i.convert('RGB')
  X_i = np.array(X_i) / 255.0

  X.append(X_i)

  label = fName.split("/")
  y_i = label[-2]

  y.append(y_i)

In [6]:
for class_label in set(y):
    count = 0
    for image_label in y:
        if class_label == image_label:
            count += 1
    print(f"The number of images in {class_label} class are: {count}")

The number of images in andesite class are: 103
The number of images in basalt class are: 130
The number of images in coal class are: 85
The number of images in granite class are: 190


In [7]:
from sklearn.preprocessing import LabelEncoder
lEncoder = LabelEncoder()
y = lEncoder.fit_transform(y)

print(set(y))
print(lEncoder.classes_)

X = np.array(X)
y = np.array(y)

print(X.shape)
print(y.shape)

{0, 1, 2, 3}
['andesite' 'basalt' 'coal' 'granite']
(508, 299, 299, 3)
(508,)


In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,
                                                    stratify=y, random_state=42)

print("X_train_shape: {}".format(X_train.shape))
print("X_test_shape: {}".format(X_test.shape))

X_train_shape: (381, 299, 299, 3)
X_test_shape: (127, 299, 299, 3)


In [9]:
mu = X_train.mean()
std = X_train.std()

X_train_std = (X_train-mu)/std # Apply feature scaling to X_train
X_test_std = (X_test-mu)/std # Apply feature scaling to X_test

In [10]:
InceptionV2=keras.applications.InceptionResNetV2(
    include_top=False,
)

219055592/219055592 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [11]:
base_model1_TL = keras.applications.InceptionResNetV2(weights='imagenet',
                                                  include_top=False)
for layer in base_model1_TL.layers:
  layer.trainabe = False

In [12]:
# Adding classifier

global_pool = keras.layers.GlobalAveragePooling2D()(base_model1_TL.output)

Layer_Normalization = keras.layers.BatchNormalization()(global_pool)
Drop_out = keras.layers.Dropout(rate=0.25)(Layer_Normalization)

output_ = keras.layers.Dense(units=4, activation='softmax')(Drop_out)


model1_TL = keras.models.Model(inputs=[base_model1_TL.input], outputs=[output_])

In [13]:
model1_TL.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, None, None, 3)  │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_203 (Conv2D)       │ (None, None, None, 32) │            864 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_203   │ (None, None, None, 32) │             96 │ conv2d_203[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_203            │ (None, None, None, 32) │              0 │ batch_normalization_2… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_204 (Conv2D)       │ (None, None, None, 32) │          9,216 │ activation_203[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_204   │ (None, None, None, 32) │             96 │ conv2d_204[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_204            │ (None, None, None, 32) │              0 │ batch_normalization_2… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_205 (Conv2D)       │ (None, None, None, 64) │         18,432 │ activation_204[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_205   │ (None, None, None, 64) │            192 │ conv2d_205[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_205            │ (None, None, None, 64) │              0 │ batch_normalization_2… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_4           │ (None, None, None, 64) │              0 │ activation_205[0][0]   │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_206 (Conv2D)       │ (None, None, None, 80) │          5,120 │ max_pooling2d_4[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_206   │ (None, None, None, 80) │            240 │ conv2d_206[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_206            │ (None, None, None, 80) │              0 │ batch_normalization_2… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_207 (Conv2D)  

 Total params: 54,349,028 (207.33 MB)

 Trainable params: 54,285,412 (207.08 MB)

 Non-trainable params: 63,616 (248.50 KB)

In [14]:
model1_TL.compile(loss='sparse_categorical_crossentropy',
                 optimizer='adam',
                 metrics=['accuracy'])

callbacks_TL = [
             keras.callbacks.ModelCheckpoint("bestTL.weights.h5",
                                             monitor='val_accuracy',
                                             save_weights_only=True,
                                             save_best_only=True)]

history_TL = model1_TL.fit(x = X_train_std, y = y_train, epochs=10,
                          validation_split=0.1, batch_size=32, callbacks=callbacks_TL)

Epoch 1/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 338s 14s/step - accuracy: 0.4816 - loss: 2.4168 - val_accuracy: 0.2821 - val_loss: 2.9523
Epoch 2/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - accuracy: 0.6912 - loss: 0.6900 - val_accuracy: 0.3333 - val_loss: 837.4117
Epoch 3/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 25s 919ms/step - accuracy: 0.8579 - loss: 0.4700 - val_accuracy: 0.1538 - val_loss: 44.5324
Epoch 4/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 21s 937ms/step - accuracy: 0.9124 - loss: 0.2478 - val_accuracy: 0.3333 - val_loss: 4.8541
Epoch 5/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 21s 975ms/step - accuracy: 0.9579 - loss: 0.1141 - val_accuracy: 0.3333 - val_loss: 1.8084
Epoch 6/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 27s 2s/step - accuracy: 0.9836 - loss: 0.0604 - val_accuracy: 0.5128 - val_loss: 2.3571
Epoch 7/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 14s 981ms/step - accuracy: 0.9736 - loss: 0.0764 - val_accuracy: 0.4103 - val_loss: 4.1930
Epoch 8/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 46s 4s/step - accuracy: 0.9816 - loss: 0.1566 - val_accuracy: 0

In [19]:
from numpy import save

save('/content/drive/MyDrive/DLPROJECT/X_train_std.npy', X_train_std)
save('/content/drive/MyDrive/DLPROJECT/X_test_std.npy', X_test_std)

save('/content/drive/MyDrive/DLPROJECT/y_train.npy', y_train)
save('/content/drive/MyDrive/DLPROJECT/y_test.npy', y_test)

In [22]:
model1_TL.load_weights("bestTL.weights.h5")

In [23]:
model1_TL.save('/content/drive/MyDrive/DLPROJECT/01_InceptionResNetV2_TransferLearning_Best_Model.keras')